In [93]:
import csv
import random

In [94]:
def loadDataset(filename, split, trainingSet, testSet):
    #with open(r'/Users/salma/Studies/MachineLearning/ml_edureka/datasets/iris_data.csv') as csvfile:
    
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        next(lines)

        dataset = list(lines)
        for x in range(len(dataset) - 1):
            for y in range(4):
                
                # Convert the flower measures that were loaded as strings into numbers
                dataset[x][y] = float(dataset[x][y])
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])  

In [95]:
### Test ###
trainingSet = []
testSet = []
loadDataset('/Users/salma/Studies/MachineLearning/ml_edureka/datasets/iris_data.csv',
           0.66, trainingSet, testSet)

print('Train: ' +repr(len(trainingSet)))
print('Test: ' +repr(len(testSet)))

Train: 101
Test: 48


#### Calculate similarity between any 2 instances so that we can locate k most similar instances in the training dataset and in turn make our prediction

#### Given that all 4 meausres are numeric and similar units --> can use Euclidean distance. Additionally we want to include which fields to include in the distance calculation

In [96]:
import math
def euclDist(inst1, inst2, length): # how many attributes to include
    dist = 0
    for x in range(length):
        dist += pow((inst1[x] - inst2[x]), 2)
    return math.sqrt(dist)

In [97]:
### Test ###
data1 = [2, 2, 2, 'a']
data2 = [4, 4, 4, 'b']

dist = euclDist(data1, data2, 3) # sqrt(3*(4-2)^2)
print ('Distance: ' +repr(dist))

Distance: 3.4641016151377544


In [98]:
### Look for KNN. Using the similarity measure find the k most
### similar instances for a given unseen instance.
### Calculating the distances for all the instances and selecting
### a subset with the smallest distance value

In [99]:
### function to return the k most similar neighbors for a given test instance
import operator
def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance) - 1
    for x in range(len(trainingSet)):
        dist = euclDist(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors=[]
    
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors
        

In [100]:
### Test ###
trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
testInstance = [5, 5, 5]
k = 1
neighbors = getNeighbors(trainSet, testInstance, 1)
print(neighbors)

[[4, 4, 4, 'b']]


In [101]:
### After locating most similar neighbor to a test instance,
### next instance is to predict a response based on their neighbors
### --> by allowing each neighbor to vote for the class attribute 
### and take majority vote as the prediction

In [102]:
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    ## iteritems renamed to items in py 3        
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1),
                        reverse=True)
    return sortedVotes[0][0]

In [103]:
### Test ###
neighbors = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
response = getResponse(neighbors)
print(response)

a


In [129]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        # initially if testSet[x][-1] == predictions[x]
        # but is wasn't validating string equality
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100

In [130]:
### Test ###
testSet = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
predictions = ['a', 'a', 'a']
accuracy = getAccuracy(testSet, predictions)
print(accuracy)

66.66666666666666


In [131]:
def main():
    # prepare data
    trainingSet = []
    testSet = []
    split = 0.67
    loadDataset('/Users/salma/Studies/MachineLearning/ml_edureka/datasets/iris_data.csv',
               split, trainingSet, testSet)
    print('Train set: ' +repr(len(trainingSet)))
    print('Test set: ' +repr(len(testSet)))
    # generate predictions
    predictions = []
    k = 3
    for x in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        #print('> predicted=' + repr(result) + ', actual=' +repr(testSet[x][-1]))
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: ' +repr(accuracy) + '%')
    
main()

Train set: 119
Test set: 30
Accuracy: 96.66666666666667%
